In [1]:
# from flask_other.app import app
from flask import Flask
from flask_socketio import SocketIO, emit,join_room,leave_room
from threading import Lock
import os,sys
# sys.path.append('../../../../Chatbot1.0/')
# sys.path.append('/home/kai/data/Chatbot1.0/API/v1/frontend/')
app=Flask("demo")
from flask import  render_template,request
import time
import re

import pandas as pd
import numpy as np
import time
import re
import requests



##################################
async_mode = None
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app, async_mode=async_mode)

thread = None
thread_lock = Lock()
clients = []

name_space = '/chat'

@app.route('/')
def index():
    return render_template('index.html', async_mode=socketio.async_mode)


@socketio.on('client_send',namespace=name_space)
def client_msg(msg):  
    print(msg['data']) # this the contentext sent by client
    response = 'data get'
    socketio.emit('my_response',{'data':response},room = request.sid, namespace=name_space)

#sned message to a specific user
def unique_message(uid):
    socketio.emit('unique_messgae',{'data':'are you still here'}, room=uid, namespace=name_space)
    
@socketio.on('disconnect',namespace=name_space)
def disconnect():
    uid = request.sid
    print(uid + " disconnect")
    clients.remove(uid)
    leave_room(uid)
    last = len(clients)
    socketio.emit('unique_message',{'data':'are you still here'}, room = clients[last-1],namespace = name_space)
    
@socketio.on('connect',namespace=name_space)
def connect():
    print('connect')
    print(request.sid)
    uid = request.sid
    clients.append(uid)
    print(len(clients))
    
    join_room(uid)
    socketio.emit('my_response',{'data':'hello'},room = uid,namespace=name_space) #the first sentence 


    



In [2]:
print('http://10.0.24.31:8899/')
print('http://0.0.0.0:6006/')
socketio.run(app,'0.0.0.0',6006)

http://10.0.24.31:8899/
http://0.0.0.0:6006/
connect
5f7364fabdba48639f786d8e1f2abcad
1



KeyboardInterrupt
2018-06-27T15:05:24Z


KeyboardInterrupt: 

In [ ]:
class Cache:
    def __init__(self):
        self.session_pool = []
        self.active_session = {}
        self.session_timer = {}
        
        
    def create_session(self, stage=1):
        available = list(set(range(1000)) - set(list(self.active_session.keys())))
        if len(available) > 0:
            sessionId = available[0]
            if stage == 1:
                self.active_session[sessionId] = TreeStage1()
                self.session_timer[sessionId] = time.time()
        else:
            sessionId = None
        return sessionId
    
    
    def purge_inactive(self):
        sec=500
        current = time.time()
        inactive_list = []
        for each in self.session_timer:
            if current - self.session_timer[each] > sec:
                print('{} session is inactive, will be removed!'.format(each))
                inactive_list.append(each)
        for key in inactive_list:
            try:
                del self.active_session[key]
            except KeyError:
                pass
            try:
                del self.session_timer[key]
            except KeyError:
                pass
            print('{} is removed'.format(key))
        gc.collect()